# Robinhood tax document to pandas

This script helps me file state taxes since there are 17 pages of dividend details and I no longer want to manually calculate the totals within each state.

Specifically, since...

- the tax software wants to know how much dividends and sales gains/losses I earned while residing in each state,
- Robinhood doesn't offer a nice, clean CSV for dividends

I wrote this script (alongside ChatGPT) to:
- crop PDF where the tables are
- converts the cropped image to text
- serializes captured rows as pandas DataFrame
- sum up dividends prior to and after move date

Use at your own risk if you decide to use this for your own taxes.

## Preprocessing

First, adjust custom settings...

Then:

- Open PDF
- Crop desired part of table
- Save as PNG

Finally:

- Open saved PNG
- Read text from PNG
- Convert text to pandas
- Join all tables
- Fix column data types

In [ ]:
import fitz
import cv2
import pytesseract
import pandas as pd

pd.set_option("display.max_columns", None)

# custom settings
pdf_file = (
    "robinhood.pdf"  # downloaded Consolidated Tax Statement from Robinhood account
)
start_page = 14  # zero based index--if PDF reader shows "Page 15 of 42", use 14
end_page = 32  # if last page of PDF reader shows "Page 32 of 42", use 32
move_date = pd.to_datetime("2022-05-24")  # date moved to a different state
tax_year = move_date.strftime("%y")

# read and crop the PDF pages to desired part of tables and save as PNG files
x0, y0, x1, y1 = 350, 85, 650, 535  # coordinates of the region to be saved
doc = fitz.open(pdf_file)
for page_number in range(start_page, end_page):
    output_image_file = f"r{page_number:02.0f}.png"
    page = doc[page_number]
    if page_number == 14:
        rect = fitz.Rect(x0, y0 + 125, x1, y1)
    else:
        rect = fitz.Rect(x0, y0, x1, y1)
    pix = page.get_pixmap(matrix=fitz.Matrix(300 / 72, 300 / 72), clip=rect)
    pix.save(output_image_file)

# read png and extract text, converting to pandas
df_list = []
for page_number in range(start_page, end_page):
    img = cv2.imread(f"r{page_number:02.0f}.png")
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    text = pytesseract.image_to_string(gray, lang="eng", config="--psm 6")
    # include only rows where there's a date available
    # e.g. checking if /22 in row because the total lines, like
    # "Total Dividends & distributions" and "Total Foreign tax withheld"
    # do not have a date in the same line
    rows = (
        row.split(maxsplit=2)
        for row in text.strip().split("\n")
        if f"/{tax_year} " in row
    )
    df = pd.DataFrame(rows, columns=["date", "amount", "transaction"])
    df_list.append(df)

# join all the tables and do some postprocessing on the column data types
df = pd.concat(df_list).set_index("date")
df["amount"] = df["amount"].astype(float)
df.index = pd.to_datetime(df.index)
print(df["transaction"].unique())


## Total dividends by move date

In [ ]:
# since all the desired transaction categories have " dividend" in them,
# exclude the "Nondividend distribution" by checking by space
# 'Qualified dividend', 'Nonqualified dividend', 'Nondividend distribution', 'Section 199A dividend'
df_sub = df.loc[df["transaction"].str.contains(" dividend")]
print(
    f"""
    Previous State Dividends: {df_sub.loc[df_sub.index < move_date, "amount"].sum():.2f},
    Current State Dividends: {df_sub.loc[df_sub.index >= move_date, "amount"].sum():.2f},
    Total Dividends: {df_sub["amount"].sum():.2f}
"""
)


## Total sales by move date

In [ ]:
# now do something similar for sales of stocks; ignore the first two rows which is a summary of dividends
# (why didn't robinhood include all details of dividends like the sales? :/)
df_csv = pd.read_csv("robinhood_transactions.csv", skiprows=2, index_col="SALE DATE", parse_dates=True)
df_csv["gain_or_loss"] = df_csv["SALES PRICE"] - df_csv["COST BASIS"]
print(
    f"""
    Previous State Gains or Losses: {df_csv.loc[df_csv.index < move_date, "gain_or_loss"].sum():.2f},
    Current State Gains or Losses: {df_csv.loc[df_csv.index >= move_date, "gain_or_loss"].sum():.2f},
    Total Gains or Losses: {df_csv["gain_or_loss"].sum():.2f}
""")